<a href="https://colab.research.google.com/github/yujinc129-oss/Drama_data_project/blob/main/03_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
drama =pd.read_json('https://github.com/yujinc129-oss/Drama_data_project/raw/refs/heads/main/data/processed/02_pre_2.json', orient="records")

In [47]:
#방영연도 컬럼추가

year = pd.read_csv('https://github.com/yujinc129-oss/Drama_data_project/raw/refs/heads/main/data/data_for_add')
aired_year = year['start airing']

In [48]:
aired_year = year[['drama_name','start airing']]

In [49]:
drama = pd.merge(drama, aired_year, left_on='drama', right_on='drama_name')

In [52]:
drama

,actor,drama,role,gender,genres,day,network,score,age,start airing
0,junghaein,snowdrop,Main Role,Male,"[romance, society, drama]","[saturday, sunday]",[JTBC],8.20,34,2021
1,junghaein,d.p.,Main Role,Male,"[etc, drama]",[friday],[NETFLIX],8.43,34,2021
2,junghaein,apieceofyourmind,Main Role,Male,"[romance, drama]","[monday, tuesday]",[TVN],7.97,33,2020
3,junghaein,onespringnight,Main Role,Male,"[romance, drama]","[thursday, wednesday]","[NETFLIX, MBC]",8.17,32,2019
4,junghaein,somethingintherain,Main Role,Male,"[romance, drama]","[friday, saturday]","[JTBC, NETFLIX]",7.92,31,2018
...,...,...,...,...,...,...,...,...,...,...
2212,kimjongkook,theproducers,Support Role,Male,"[romance, comedy, drama]","[friday, saturday]",[KBS],7.64,40,2015
2213,jungyoonhak,myunfortunateboyfriend,Main Role,Male,"[romance, drama]","[friday, saturday]",[MBC],7.54,32,2015
2214,jungjoonyoung,thelover,Main Role,Male,"[romance, comedy, drama]",[thursday],[ETC],7.80,27,2015
2215,kimjoon,cityofthesun,Main Role,Male,"[romance, society, drama, thriller]","[friday, saturday]",[MBC],7.70,32,2015


In [53]:
#주연비율 파생변수 생성
drama_cnt = drama.actor.value_counts(sort=False)
main_cnt =drama[drama['role']=='Main Role'].groupby('actor',sort=False,)['drama'].count()
main_cnt = main_cnt.fillna(0)
drama_cnt = drama_cnt.fillna(0)
ratio = main_cnt / drama_cnt
ratio = ratio.replace([np.inf, -np.inf], np.nan).fillna(0)

ratio =ratio.reset_index()

drama = pd.merge(drama, ratio, how='left')

drama.rename(columns={0:'main_ratio'},inplace=True)

drama

,actor,drama,role,gender,genres,day,network,score,age,start airing,main_ratio
0,junghaein,snowdrop,Main Role,Male,"[romance, society, drama]","[saturday, sunday]",[JTBC],8.20,34,2021,0.75
1,junghaein,d.p.,Main Role,Male,"[etc, drama]",[friday],[NETFLIX],8.43,34,2021,0.75
2,junghaein,apieceofyourmind,Main Role,Male,"[romance, drama]","[monday, tuesday]",[TVN],7.97,33,2020,0.75
3,junghaein,onespringnight,Main Role,Male,"[romance, drama]","[thursday, wednesday]","[NETFLIX, MBC]",8.17,32,2019,0.75
4,junghaein,somethingintherain,Main Role,Male,"[romance, drama]","[friday, saturday]","[JTBC, NETFLIX]",7.92,31,2018,0.75
...,...,...,...,...,...,...,...,...,...,...,...
2212,kimjongkook,theproducers,Support Role,Male,"[romance, comedy, drama]","[friday, saturday]",[KBS],7.64,40,2015,0.00
2213,jungyoonhak,myunfortunateboyfriend,Main Role,Male,"[romance, drama]","[friday, saturday]",[MBC],7.54,32,2015,1.00
2214,jungjoonyoung,thelover,Main Role,Male,"[romance, comedy, drama]",[thursday],[ETC],7.80,27,2015,1.00
2215,kimjoon,cityofthesun,Main Role,Male,"[romance, society, drama, thriller]","[friday, saturday]",[MBC],7.70,32,2015,1.00


In [55]:
#플랫폼 등급 컬럼 추가
# 1. 등급 매핑 딕셔너리 정의 (숫자가 작을수록 높은 등급)
tier_map = {'NETFLIX':1, 'TVN':1, 'JTBC':1, 'KBS':2, 'MBC':2, 'SBS':2, 'OCN':2, 'ETC':3}
default_tier = 3

# 2. 가장 높은 등급을 찾아주는 함수 정의
def get_highest_tier(platform_list):
    # 리스트에 있는 각 플랫폼의 등급을 찾음
    # tier_map에 없는 플랫폼은 default_tier로 처리
    tiers = [tier_map.get(p, default_tier) for p in platform_list]

    # 등급 리스트가 비어있으면 기본 등급 반환
    if not tiers:
        return default_tier

    # 가장 작은 숫자(가장 높은 등급)를 반환
    return min(tiers)


# 3. apply 함수를 사용하여 'platform_tier' 컬럼 생성
drama['platform_tier'] = drama['network'].apply(get_highest_tier)

drama

,actor,drama,role,gender,genres,day,network,score,age,start airing,main_ratio,platform_tier
0,junghaein,snowdrop,Main Role,Male,"[romance, society, drama]","[saturday, sunday]",[JTBC],8.20,34,2021,0.75,1
1,junghaein,d.p.,Main Role,Male,"[etc, drama]",[friday],[NETFLIX],8.43,34,2021,0.75,1
2,junghaein,apieceofyourmind,Main Role,Male,"[romance, drama]","[monday, tuesday]",[TVN],7.97,33,2020,0.75,1
3,junghaein,onespringnight,Main Role,Male,"[romance, drama]","[thursday, wednesday]","[NETFLIX, MBC]",8.17,32,2019,0.75,1
4,junghaein,somethingintherain,Main Role,Male,"[romance, drama]","[friday, saturday]","[JTBC, NETFLIX]",7.92,31,2018,0.75,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2212,kimjongkook,theproducers,Support Role,Male,"[romance, comedy, drama]","[friday, saturday]",[KBS],7.64,40,2015,0.00,2
2213,jungyoonhak,myunfortunateboyfriend,Main Role,Male,"[romance, drama]","[friday, saturday]",[MBC],7.54,32,2015,1.00,2
2214,jungjoonyoung,thelover,Main Role,Male,"[romance, comedy, drama]",[thursday],[ETC],7.80,27,2015,1.00,3
2215,kimjoon,cityofthesun,Main Role,Male,"[romance, society, drama, thriller]","[friday, saturday]",[MBC],7.70,32,2015,1.00,2


In [ ]:
#감독 작가 파생변수 생성 -ing
# director = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_set/DRAMA_PROJECT/etc/drama/korean_drama.csv')
# director[['drama_name','director','screenwriter']]

In [56]:
drama.to_json("03_feature", orient="records", indent=2)